In [ ]:
# 커널 파이썬3.10사용
%pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# EUC-KR 인코딩으로 파일 읽기
df = pd.read_csv("./서울시  대기오염도 정보.csv", encoding='euc-kr')

df.head()


In [ ]:
selected_locations = ['강동구', '강서구', '강남구', '강북구', '강남구','강동구','강북구','강서구','관악구','관악산','광진구','구로구','금천구','노원구','도봉구',
                      '동대문구','동작구','마포구','북한산','서대문구','서초구','성북구','송파구','양천구','영등포구','용산구','은평구','종로','종로구','중구','중랑구']

# 선택한 지역들 중 하나인 행들을 선택
df_selected = df[df['측정소명'].isin(selected_locations)]

# 오존농도(ppm)이 0보다 큰 행들만 선택
df = df[(df['오존농도(ppm)'] > 0)]

# 각 지역의 평균 오존농도 계산
top = df.groupby('측정소명')['오존농도(ppm)'].mean().sort_values(ascending=False)
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'  # 사용할 한글 폰트 설정

# 시각화
plt.figure(figsize=(10,7))
sns.barplot(x=top.index, y=top.values, palette='viridis')

plt.title('지역에 따른 평균 오존농도(ppm)', color='blue')
plt.xlabel('지역')
plt.ylabel('평균 오존농도(ppm)')
plt.xticks(rotation=45)  # x축 라벨을 45도 회전하여 보기 편하게 설정
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# 클러스터링에 사용할 열 선택
X = df[['오존농도(ppm)', '측정소명']]

# LabelEncoder를 사용하여 '측정소명' 열을 숫자형으로 변환
le = LabelEncoder()
X['측정소명'] = le.fit_transform(X['측정소명'])

# KMeans 클러스터링 수행
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(X)

# 클러스터링 결과 시각화
plt.figure(figsize=(10, 7))
sns.scatterplot(data=X, x='측정소명', y='오존농도(ppm)', hue=kmeans.labels_, palette='Set1', s=100)
plt.xticks(rotation=45)
plt.title('오존 농도를 기반으로 한 클러스터링')
plt.xlabel('측정소명 (숫자형)')
plt.ylabel('오존농도 (ppm)')
plt.legend(title='Cluster')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

nclusters = 3  # 클러스터 개수
seed = 0  # 랜덤 시드 설정

# KMeans 모델 초기화
km = KMeans(n_clusters=nclusters, random_state=seed, n_init=10)

# 데이터에 모델 적합
km.fit(X)

# 각 데이터 포인트의 클러스터 예측
y_cluster_kmeans = km.predict(X)
y_cluster_kmeans


In [ ]:
from sklearn import metrics

#metrics.silhouette_score 함수는 실루엣 점수를 계산하는 함수.  이 점수가 높을수록 클러스터링 결과가 좋다고 해석할 수 있다.
# 클러스터링 결과의 실루엣 점수 계산
score = metrics.silhouette_score(X, y_cluster_kmeans)

# 출력
score


In [ ]:
from sklearn.cluster import KMeans

# 각 클러스터 수에 대한 WCSS를 저장할 리스트 초기화
wcss = []

# 클러스터 수를 1부터 10까지 변경하면서 K-means 알고리즘 실행
for i in range(1, 11):
    # K-means 알고리즘 생성 및 학습
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42, n_init=10)
    kmeans.fit(X)
    
    # WCSS 값을 리스트에 추가
    wcss.append(kmeans.inertia_)


In [ ]:
# 그래프의 크기를 설정
plt.figure(figsize=(10,5))

# range(1, 11)에 대한 WCSS 값을 선 그래프로 그림
sns.lineplot(data=(range(1, 11), wcss), marker='o', color='red')
plt.title('Elbow')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# KMeans 클러스터링에 사용할 열 선택
X = df[['오존농도(ppm)']]

# KMeans 모델 생성 및 학습
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(X)

# 클러스터링 결과 예측
labels = kmeans.labels_

# 클러스터링 결과 시각화
plt.figure(figsize=(10, 6))  # 플롯 사이즈 조정
plt.scatter(df['측정소명'], df['오존농도(ppm)'], c=labels, cmap='viridis')
plt.xticks(rotation=45, ha='right')  # xticks 회전 및 정렬 방식 설정
plt.xlabel('측정소명')
plt.ylabel('오존농도(ppm)')
plt.title('Clustering based on Ozone Concentration')
plt.colorbar(label='Cluster')
plt.show()


In [ ]:
# 필요한 모듈을 임포트
from sklearn.metrics import accuracy_score
import pandas as pd

# majority_vote 함수를 정의
# 이 함수는 각 클러스터에서 가장 많이 등장하는 측정소명을 찾아서 해당 측정소명을 예측값으로 사용
def majority_vote(cluster_labels, true_labels):
    # 각 클러스터에 대한 예측값을 저장할 딕셔너리를 초기화
    cluster_to_label = {}
    # 모든 클러스터에 대해 반복하여 예측값을 결정
    for cluster in set(cluster_labels):
        # 해당 클러스터에 속하는 인덱스를 찾습니다.
        cluster_indices = cluster_labels == cluster
        # 해당 클러스터에서 가장 많이 등장하는 값(측정소명)을 찾습니다.
        major_label = true_labels[cluster_indices].mode()[0]
        # 클러스터와 해당 클러스터의 예측값을 딕셔너리에 저장
        cluster_to_label[cluster] = major_label
    # 각 데이터 포인트에 대해 예측값을 생성
    predicted_labels = [cluster_to_label[cluster] for cluster in cluster_labels]
    return predicted_labels

# '측정소명'을 숫자로 인코딩한 열을 추가
df['측정소명_code'] = le.fit_transform(df['측정소명'])

# 정답 레이블을 생성
y_true = df['측정소명_code']

# majority_vote 함수를 사용하여 예측값을 계산
predicted_labels = majority_vote(labels, y_true)

# 정확도를 계산
accuracy = accuracy_score(y_true, predicted_labels)
print(f"Accuracy: {accuracy:.2f}")

